<a href="https://colab.research.google.com/github/ravindrabharathi/AVS-provisioning-Java/blob/master/044-rb-Get_Labels_for_patent_doc_with_missing_cpc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Install and import modules 

In [1]:
!pip install pysolr

     |████████████████████████████████| 55 kB 2.2 MB/s 
  Created wheel for pysolr: filename=pysolr-3.9.0-py2.py3-none-any.whl size=19690 sha256=8b19cf6b00614dcf1402c7353ffdc329668f82b9a668317909b95581a72eb730
  Stored in directory: /root/.cache/pip/wheels/07/39/f8/fa031d35471643f5a0c5847be457342b1262c019756408874b
Successfully built pysolr


In [2]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 4.2 MB/s 
     |████████████████████████████████| 3.3 MB 29.4 MB/s 
     |████████████████████████████████| 895 kB 50.7 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 50.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install sentence-transformers

     |████████████████████████████████| 78 kB 2.7 MB/s 
     |████████████████████████████████| 1.2 MB 14.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=0360fc04a1e3e2b31ec3d172880237d6cf3155abb80b7b748b51a1eae9f2357b
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [4]:
import json 
import pandas as pd
import numpy as np
import pysolr
import bs4 as bs
import urllib.request
import plotly.express as px

In [5]:
import warnings
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
import torch

In [6]:
from sentence_transformers import SentenceTransformer

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from collections import OrderedDict

In [8]:
import pprint
pp=pprint.PrettyPrinter(indent=2)

### solr config 

In [9]:
user=  "picampus" #@param {type:"string"}
pwd = "a!5chool.2021" #@param {type:"string"}
auth = (user,pwd)


In [10]:
core_names=["patents-ep"]
facet_field='cpc_classifications'

In [11]:
solr = pysolr.Solr("http://35.223.230.190:8983/solr/",auth=auth)


### solr data parsing functions

In [12]:
mincount=10000

In [13]:
def getData(q,solr,core_name,field_name):
  # http://35.223.230.190:8983/solr/patents-ep/select?facet.field=cpc_classifications&facet=on&indent=on&q=*:*&rows=0&wt=json

  

  qStr="/"+core_name+"/select?indent=off&q="+q+"&wt=json&&rows=0&facet=on&facet.limit=-1&facet.mincount="+str(mincount)+"&facet.field="+field_name
  
  #qStr="/"+core_name+"/select?indent=off&q="+q+"&wt=json&fl=start_date&rows=0&facet.range=start_date&facet=on&f.start_date.facet.range.start=1960-01-01T00:00:00Z&f.start_date.facet.range.end=2022-01-01T00:00:00Z&f.start_date.facet.range.gap=%2B1MONTH"
  data=solr._send_request("GET",qStr)
  jsonData=json.loads(data)
  return jsonData

def getData2(q,solr,core_name,data_fields,nrows):
  

  #http://35.223.230.190:8983/solr/patents-ep/select?fl=invention_title_en&indent=on&q=cpc_classifications:%22A61P%2043/00%22&rows=1000&wt=json

  qStr="/"+core_name+"/select?indent=off&q="+q+"&wt=json&&rows="+str(nrows)+"&fl="+data_fields
  
  #qStr="/"+core_name+"/select?indent=off&q="+q+"&wt=json&fl=start_date&rows=0&facet.range=start_date&facet=on&f.start_date.facet.range.start=1960-01-01T00:00:00Z&f.start_date.facet.range.end=2022-01-01T00:00:00Z&f.start_date.facet.range.gap=%2B1MONTH"
  data=solr._send_request("GET",qStr)
  jsonData=json.loads(data)
  return jsonData  

def getDocs_missing(q,solr,core_name,data_fields,nrows):
  qStr="/"+core_name+"/select?indent=off&q="+q+"&wt=json&&rows="+str(nrows)+"&fl="+data_fields
  
  #qStr="/"+core_name+"/select?indent=off&q="+q+"&wt=json&fl=start_date&rows=0&facet.range=start_date&facet=on&f.start_date.facet.range.start=1960-01-01T00:00:00Z&f.start_date.facet.range.end=2022-01-01T00:00:00Z&f.start_date.facet.range.gap=%2B1MONTH"
  data=solr._send_request("GET",qStr)
  jsonData=json.loads(data)
  return jsonData 





###get titles for documents that are missing source_asjc_codes - lets say 20 documents  

In [14]:
jsonDocs=getData2('-cpc_classifications:*',solr,core_names[0],'id,invention_title_en',20) 
  
missing_df=pd.DataFrame(jsonDocs["response"]["docs"])

In [15]:
missing_df.head()

,id,invention_title_en
0,EPB1/2574204,AIRFIELD LUMINAIRE HAVING REMOVABLE OPTICAL PO...
1,EPA1/0578632,LADY'S SHOE WITH MISSING SOLE SECTION.
2,EPA1/0578699,GALVANISATION DEVICE FOR PLATE-LIKE WORKPIECES...
3,EPA1/0579598,PERMANENT MAGNET ASSEMBLY.
4,EPA1/0582564,PORTABLE NMR AND NQR SPECTROMETERS.


##Define model

In [16]:
#model=SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# use model from here https://huggingface.co/sentence-transformers/allenai-specter

model=SentenceTransformer('sentence-transformers/allenai-specter')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/622 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/462k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/331 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Mount Google Drive

In [17]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


Inspect embeddings directory where papers and patent embeddings were saved

In [18]:
!ls '/gdrive/MyDrive/PISCHOOL/embeddings/'

lens_papers


Get a list of embedding files for lens_papers core 

In [22]:
import os 
fldr='/gdrive/MyDrive/PISCHOOL/embeddings/patents-ep/cpc_classifications/'
embedding_list=os.listdir(fldr)

Form a dictionary to hold asjc class as key and corresponding embeddings 

In [24]:
embedding_dict={}
for fname in embedding_list:
  cpc_class=fname.split('.')[0]
  cpc_class=cpc_class.replace("_1_","/")
  cpc_class=cpc_class.replace('"','')
  print('loading '+fname)
  embedding_dict[cpc_class]=np.mean(np.load(fldr+fname),axis=0)

loading "A23V2002_1_00.npy
loading "A61K 38_1_00.npy
loading "A61K 39_1_00.npy
loading "A61K 45_1_06.npy
loading "A61K 47_1_10.npy
loading "A61K 48_1_00.npy
loading "A61K 9_1_0014.npy
loading "A61K 9_1_0019.npy
loading "A61K2039_1_505.npy
loading "A61P 1_1_00.npy
loading "A61P 1_1_02.npy
loading "A61P 1_1_04.npy
loading "A61P 1_1_16.npy
loading "A61P 11_1_00.npy
loading "A61P 11_1_06.npy
loading "A61P 13_1_02.npy
loading "A61P 13_1_12.npy
loading "A61P 15_1_00.npy
loading "A61P 17_1_00.npy
loading "A61P 17_1_02.npy
loading "A61P 17_1_06.npy
loading "A61P 19_1_00.npy
loading "A61P 19_1_02.npy
loading "A61P 19_1_08.npy
loading "A61P 19_1_10.npy
loading "A61P 21_1_00.npy
loading "A61P 25_1_00.npy
loading "A61P 25_1_02.npy
loading "A61P 25_1_04.npy
loading "A61P 25_1_06.npy
loading "A61P 25_1_08.npy
loading "A61P 25_1_14.npy
loading "A61P 25_1_16.npy
loading "A61P 25_1_18.npy
loading "A61P 25_1_20.npy
loading "A61P 25_1_22.npy
loading "A61P 25_1_24.npy
loading "A61P 25_1_28.npy
loading "A6

In [ ]:
#pp.pprint(embedding_dict)

Get embeddings for the 20 documents with missing cpc classes 

In [25]:


missing_df['embeddings']=model.encode(missing_df['invention_title_en'].str.lower().tolist(),batch_size=100,show_progress_bar=True,device='cuda',convert_to_numpy=True).tolist()


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
missing_df.head(20)

,id,invention_title_en,embeddings
0,EPB1/2574204,AIRFIELD LUMINAIRE HAVING REMOVABLE OPTICAL PO...,"[-0.9897760152816772, -0.6216489672660828, 0.1..."
1,EPA1/0578632,LADY'S SHOE WITH MISSING SOLE SECTION.,"[-0.7814669609069824, -0.07528863847255707, 0...."
2,EPA1/0578699,GALVANISATION DEVICE FOR PLATE-LIKE WORKPIECES...,"[-0.015425599180161953, 0.7762120962142944, 0...."
3,EPA1/0579598,PERMANENT MAGNET ASSEMBLY.,"[-0.9263264536857605, 0.5293582677841187, 0.41..."
4,EPA1/0582564,PORTABLE NMR AND NQR SPECTROMETERS.,"[-0.6532235741615295, 0.4677494466304779, 0.20..."
5,EPA1/0584278,PYROMETER TEMPERATURE MEASUREMENT OF PLURAL WA...,"[0.4335489273071289, -0.7825362086296082, -0.5..."
6,EPA1/0585221,ANAEROBIC DIGESTION PROCESS.,"[-0.4933181405067444, 0.5446655750274658, -0.0..."
7,EPA1/0585223,TRANSPORTABLE LIQUID PRODUCTS CONTAINER.,"[-0.5816268920898438, 0.15479065477848053, -0...."
8,EPA1/0586444,IMAGE SYNTHESIS AND PROCESSING.,"[-0.15519924461841583, 0.6634081602096558, -0...."
9,EPA1/0580770,"PRASEODYMIUM-PALLADIUM BINARY OXIDE, CATALYST ...","[0.10568001866340637, 0.1419169306755066, 0.60..."


form a numpy array of the document embeddings so we can calculate cosine similarity against class embeddings 

In [27]:
missing_embeddings_list=missing_df['embeddings'].tolist()

In [28]:
missing_array=np.array(missing_df['embeddings'].to_list())

Calculate cosine similarity between the 20 document embeddings against asjc code embeddings and store the top 10 similar asjc codes with similarity score 

In [29]:
results=[]
results2=[]

for i in range(20):
  similarity_dict={}
  a=missing_array[i].reshape(1,-1)
  for key,val in embedding_dict.items():
    similarity_dict[key]=cosine_similarity(a,val.reshape(1,-1))
  results.append(list(OrderedDict(sorted(similarity_dict.items(), key=lambda x: x[1], reverse=True)).items() )[:10])
  results2.append(list(OrderedDict(sorted(similarity_dict.items(), key=lambda x: x[1], reverse=True)) )[:10])
pp.pprint(results)

[ [ ('F21Y2115/10', array([[0.8498293]])),
    ('Y10T 29/49826', array([[0.78688625]])),
    ('Y02E 10/50', array([[0.75368528]])),
    ('B29C 66/71', array([[0.74942755]])),
    ('Y02T 50/60', array([[0.74763291]])),
    ('B29C 66/1122', array([[0.74711429]])),
    ('H01L2224/48091', array([[0.74266527]])),
    ('B32B 27/36', array([[0.74153118]])),
    ('B32B 7/12', array([[0.74149921]])),
    ('B32B 27/08', array([[0.73636968]]))],
  [ ('Y10T 29/49826', array([[0.74319464]])),
    ('B29C 66/71', array([[0.73822028]])),
    ('B29C 66/1122', array([[0.73610047]])),
    ('B32B 7/12', array([[0.73186996]])),
    ('B32B 27/32', array([[0.72426453]])),
    ('B32B 27/08', array([[0.72300904]])),
    ('B32B 27/36', array([[0.72205975]])),
    ('A61Q 19/00', array([[0.71324335]])),
    ('A61P 1/02', array([[0.71252246]])),
    ('A23V2002/00', array([[0.70597724]]))],
  [ ('B29C 66/1122', array([[0.83442149]])),
    ('B32B 7/12', array([[0.82895616]])),
    ('B29C 66/71', array([[0.82733416]]

append the top 10 class information to existing dataframe and print it 

In [30]:
missing_df['top-10-classes-scores']=results
missing_df['top-10-classes']=results2

In [31]:
missing_df.head(20)

,id,invention_title_en,embeddings,top-10-classes-scores,top-10-classes
0,EPB1/2574204,AIRFIELD LUMINAIRE HAVING REMOVABLE OPTICAL PO...,"[-0.9897760152816772, -0.6216489672660828, 0.1...","[(F21Y2115/10, [[0.8498293]]), (Y10T 29/49826,...","[F21Y2115/10, Y10T 29/49826, Y02E 10/50, B29C ..."
1,EPA1/0578632,LADY'S SHOE WITH MISSING SOLE SECTION.,"[-0.7814669609069824, -0.07528863847255707, 0....","[(Y10T 29/49826, [[0.74319464]]), (B29C 66/71,...","[Y10T 29/49826, B29C 66/71, B29C 66/1122, B32B..."
2,EPA1/0578699,GALVANISATION DEVICE FOR PLATE-LIKE WORKPIECES...,"[-0.015425599180161953, 0.7762120962142944, 0....","[(B29C 66/1122, [[0.83442149]]), (B32B 7/12, [...","[B29C 66/1122, B32B 7/12, B29C 66/71, B32B 27/..."
3,EPA1/0579598,PERMANENT MAGNET ASSEMBLY.,"[-0.9263264536857605, 0.5293582677841187, 0.41...","[(B82Y 10/00, [[0.82202779]]), (Y10T 29/49826,...","[B82Y 10/00, Y10T 29/49826, H01L2224/48091, H0..."
4,EPA1/0582564,PORTABLE NMR AND NQR SPECTROMETERS.,"[-0.6532235741615295, 0.4677494466304779, 0.20...","[(B82Y 10/00, [[0.81198299]]), (C12Q2600/156, ...","[B82Y 10/00, C12Q2600/156, F21Y2115/10, C12Q 1..."
5,EPA1/0584278,PYROMETER TEMPERATURE MEASUREMENT OF PLURAL WA...,"[0.4335489273071289, -0.7825362086296082, -0.5...","[(H01L2924/0002, [[0.77978176]]), (B82Y 10/00,...","[H01L2924/0002, B82Y 10/00, H01L2224/48091, B8..."
6,EPA1/0585221,ANAEROBIC DIGESTION PROCESS.,"[-0.4933181405067444, 0.5446655750274658, -0.0...","[(Y02A 50/20, [[0.84710158]]), (Y02P 10/20, [[...","[Y02A 50/20, Y02P 10/20, A23V2002/00, Y02T 10/..."
7,EPA1/0585223,TRANSPORTABLE LIQUID PRODUCTS CONTAINER.,"[-0.5816268920898438, 0.15479065477848053, -0....","[(Y10T 29/49826, [[0.74579262]]), (B29C 66/112...","[Y10T 29/49826, B29C 66/1122, B29C 66/71, B32B..."
8,EPA1/0586444,IMAGE SYNTHESIS AND PROCESSING.,"[-0.15519924461841583, 0.6634081602096558, -0....","[(H04N 19/176, [[0.89487353]]), (H04N 19/61, [...","[H04N 19/176, H04N 19/61, G06F 3/0482, G06F 3/..."
9,EPA1/0580770,"PRASEODYMIUM-PALLADIUM BINARY OXIDE, CATALYST ...","[0.10568001866340637, 0.1419169306755066, 0.60...","[(B82Y 30/00, [[0.85903441]]), (Y02P 10/20, [[...","[B82Y 30/00, Y02P 10/20, C08F 10/00, Y02P 70/5..."
